In [72]:
import pandas as pd
import requests
import json
import sys
#put root in path
sys.path.append('../')
from src.data import helpers
from src.data import faers_translations
from copy import deepcopy
import os
from dotenv import load_dotenv
load_dotenv()
import time

In [2]:
os.environ['fda_api_key']

'AxJ349dp7cWgZrO4dYC7mk3M4cTqHqa4n375Fao0'

In [3]:
faersApi = helpers.FaersApi()

# Get Data
Take top 10 interactions for each drug in list, get top 100 patient.reaction.reactionmeddrapt for them.
Take top 10 patient.reaction.reactionmeddrapt for each drug in list, get top 1000 interactions for each of them.

[best selling a-z drugs](https://www.biopharmadive.com/news/astrazeneca-pharma-dive-awards/566229/#:~:text=Lung%20cancer%20drug%20Tagrisso%20has,overtaking%20Symbicort%20earlier%20this%20year.)


In [4]:
a_z_drugs = ['symbicort','tagrisso','nexium','crestor','farxiga','brilinta','pulmicort','faslodex','zoladex','seloken','toprol-xl']
A_Z_drugs = [drug.upper() for drug in a_z_drugs]
A_Z_drugs

['SYMBICORT',
 'TAGRISSO',
 'NEXIUM',
 'CRESTOR',
 'FARXIGA',
 'BRILINTA',
 'PULMICORT',
 'FASLODEX',
 'ZOLADEX',
 'SELOKEN',
 'TOPROL-XL']

### Data Collection
In order to quickly acquire data to explore and manipulate, I created a class to enable easy interaction with the API for my purposes. This is available [here](https://github.com/cshwery/faers_ml/blob/master/src/data/helpers.py). In particular I made a recursive method (.return_categorical_counts) that takes as input a set of parameters and a set of categories and the values they take, and recursively makes api calls for all combination of category states (while also leveraging the input parameters). This allows me to quickly create a dataset, following the following design:
* Iterate through the drugs listed above
    * For each drug, identify the 10 most common patient reactions
        * For each reaction get the top 1000 additional interacting drugs
        * Do this for all combinations of age group, gender, and reaction outcome group
    * For each drug, identify the 10 most common interacting drugs
        * For each interaction, get the top 1000 reactions
        * Do this for all combinations of age group, gender, and reaction outcome group
    * Deduplicate results


In [79]:
params_1000 = {'search': {'receivedate': '[20040101+TO+20201024]'}#,'patient.drug.medicinalproduct':'symbicort'.upper()}
               #, 'count':'patient.drug.medicinalproduct'
               ,'limit':1000
              }
params_10 = {'search': {'receivedate': '[20040101+TO+20201024]'}#,'patient.drug.medicinalproduct':'symbicort'.upper()}
             #, 'count':'patient.drug.medicinalproduct'
             ,'limit':10
            }
params_10_interacts = {'search': {'receivedate': '[20040101+TO+20201024]'}#,'patient.drug.medicinalproduct':'symbicort'.upper()}
                       , 'count':'patient.drug.medicinalproduct.exact'
                       , 'limit':11
                      } #11 because drug will be listed in the results
params_10_reacts = {'search': {'receivedate': '[20040101+TO+20201024]'}#,'patient.drug.medicinalproduct':'symbicort'.upper()}
                    , 'count':'patient.reaction.reactionmeddrapt.exact'
                    , 'limit':10
                   }
params_serious = {'search': {'receivedate': '[20040101+TO+20201024]'}#,'patient.drug.medicinalproduct':'symbicort'.upper()}
                  , 'count':'serious'
                 }

In [66]:
params_10 = {'search': {'receivedate': '[20040101+TO+20201024]'}#,'patient.drug.medicinalproduct':'symbicort'.upper()}
             #, 'count':'patient.drug.medicinalproduct'
             ,'limit':10
            }
params_10_interacts = {'search': {'receivedate': '[20040101+TO+20201024]'}#,'patient.drug.medicinalproduct':'symbicort'.upper()}
                       , 'count':'patient.drug.medicinalproduct.exact'
                       , 'limit':11
                      } #11 because drug will be listed in the results
params_10_reacts = {'search': {'receivedate': '[20040101+TO+20201024]'}#,'patient.drug.medicinalproduct':'symbicort'.upper()}
                    , 'count':'patient.reaction.reactionmeddrapt.exact'
                    , 'limit':10
                   }
params_serious = {'search': {'receivedate': '[20040101+TO+20201024]'}#,'patient.drug.medicinalproduct':'symbicort'.upper()}
                  , 'count':'serious'
                 }

{'search': {'receivedate': '[20040101+TO+20201024]'},
 'count': 'patient.reaction.reactionmeddrapt.exact',
 'limit': 10}

In [20]:
f_t = copy(faers_translations)
del f_t['patient.patientagegroup']
f_t

{'patient.patientsex': {'1': 'Male', '2': 'Female'},
 'serious': {'1': 'Serious Adverse Event', '2': 'Non-Serious'}}

In [86]:
a_z_drugs = ['symbicort','tagrisso','nexium','crestor','farxiga','brilinta','pulmicort','faslodex','zoladex','seloken','toprol-xl']
A_Z_drugs = [drug.upper() for drug in a_z_drugs]
A_Z_drugs

def pause_if_required(query_count):
    "Simple function to avoid fda api 240 requests/minute rate limit"
    if query_count >= 220:
        print('pausing for 60 seconds')
        time.sleep(60)
        return 0
    else:
        return query_count

failed_drug_queries = []
query_count = 0
initialized = False
for initial_drug in A_Z_drugs:
    #Get top 10 interactions
    _params_10_interacts_ = deepcopy(params_10_interacts)
    _params_10_interacts_['search'].update({'patient.drug.medicinalproduct':initial_drug})
    #print(params_10_interacts)
    try:
        query_count = pause_if_required(query_count)
        initial_results = faersApi.make_call(_params_10_interacts_).json()['results']
        query_count +=1
        print(f'fetching co-occuring drug data for {initial_drug}')
    except KeyError:
        # This occurs when a search for a given drug fails to return any results
        failed_drug_queries.append(initial_drug)
        print(f'{initial_drug} query did not return results')
        query_count +=1
        continue
    for interaction_drug in [res['term'] for res in initial_results]:
        _params_10_reacts_ = deepcopy(params_10_reacts)
        _params_10_reacts_['search'].update({'patient.drug.medicinalproduct':initial_drug,
                                             'patient.drug.medicinalproduct.exact':interaction_drug})
        try:
            query_count = pause_if_required(query_count)
            interaction_results = faersApi.make_call(_params_10_reacts_).json()['results']
            query_count += 1
            print(f'\tfetching reactions data for ({initial_drug},{interaction_drug})')
        except KeyError:
            failed_drug_queries.append((initial_drug,interaction_drug))
            print(f'({initial_drug},{interaction_drug}) query did not return results')
            query_count +=1
            continue
        for reaction in [res['term'] for res in interaction_results]:
            init_int_react_params = deepcopy(params_serious)
            init_int_react_params['search'].update({'patient.drug.medicinalproduct':initial_drug,
                                          'patient.drug.medicinalproduct.exact':interaction_drug,
                                          'patient.reaction.reactionmeddrapt.exact':reaction})
            try:
                query_count = pause_if_required(query_count)
                init_int_react_results = faersApi.make_call(init_int_react_params).json()['results']
                query_count += 1
                print(f'\t\tfetching init/int/react data for {init_int_react_params}')
            except KeyError:
                query_count +=1
                # Try using non-exact reactions
                init_int_react_params = deepcopy(params_serious)
                init_int_react_params['search'].update({'patient.drug.medicinalproduct':initial_drug,
                                                        'patient.drug.medicinalproduct.exact':interaction_drug,
                                                        'patient.reaction.reactionmeddrapt':reaction})
                try:
                    query_count = pause_if_required(query_count)
                    init_int_react_results = faersApi.make_call(init_int_react_params).json()['results']
                    query_count += 1
                    print(f'\t\tfetching non-exact init/int/react data for ({init_int_react_params})')
                except KeyError:
                    failed_drug_queries.append((initial_drug,interaction_drug,reaction))
                    print(f'({initial_drug},{interaction_drug},{reaction}) query did not return results')
                    query_count += 1
                    continue
            # Add the query specific data to the results
            for res in init_int_react_results:
                res.update({'initial_drug':initial_drug,'interaction_drug':interaction_drug,'reaction':reaction})
            if not initialized:
                # make a pd.DataFrame first time, subsequently create temporary df and concatnate to main_df
                df = pd.DataFrame.from_records(init_int_react_results)
                initialized = True
            else:
                df = pd.concat([df,
                                pd.DataFrame.from_records(init_int_react_results)])
    

pausing for 60 seconds
fetching co-occuring drug data for SYMBICORT
	fetching reactions data for (SYMBICORT,SYMBICORT)
		fetching init/int/react data for {'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'SYMBICORT', 'patient.drug.medicinalproduct.exact': 'SYMBICORT', 'patient.reaction.reactionmeddrapt.exact': 'DYSPNOEA'}, 'count': 'serious'}
		fetching init/int/react data for {'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'SYMBICORT', 'patient.drug.medicinalproduct.exact': 'SYMBICORT', 'patient.reaction.reactionmeddrapt.exact': 'ASTHMA'}, 'count': 'serious'}
		fetching init/int/react data for {'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'SYMBICORT', 'patient.drug.medicinalproduct.exact': 'SYMBICORT', 'patient.reaction.reactionmeddrapt.exact': 'COUGH'}, 'count': 'serious'}
		fetching non-exact init/int/react data for ({'search': {'receivedate': '[20040101+TO+20201024]', '

		fetching init/int/react data for {'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'SYMBICORT', 'patient.drug.medicinalproduct.exact': 'PREDNISONE.', 'patient.reaction.reactionmeddrapt.exact': 'DYSPNOEA'}, 'count': 'serious'}
		fetching init/int/react data for {'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'SYMBICORT', 'patient.drug.medicinalproduct.exact': 'PREDNISONE.', 'patient.reaction.reactionmeddrapt.exact': 'ASTHMA'}, 'count': 'serious'}
		fetching init/int/react data for {'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'SYMBICORT', 'patient.drug.medicinalproduct.exact': 'PREDNISONE.', 'patient.reaction.reactionmeddrapt.exact': 'COUGH'}, 'count': 'serious'}
		fetching init/int/react data for {'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'SYMBICORT', 'patient.drug.medicinalproduct.exact': 'PREDNISONE.', 'patient.reaction.reactio

		fetching init/int/react data for {'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'SYMBICORT', 'patient.drug.medicinalproduct.exact': 'OMEPRAZOLE.', 'patient.reaction.reactionmeddrapt.exact': 'FATIGUE'}, 'count': 'serious'}
		fetching init/int/react data for {'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'SYMBICORT', 'patient.drug.medicinalproduct.exact': 'OMEPRAZOLE.', 'patient.reaction.reactionmeddrapt.exact': 'COUGH'}, 'count': 'serious'}
		fetching init/int/react data for {'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'SYMBICORT', 'patient.drug.medicinalproduct.exact': 'OMEPRAZOLE.', 'patient.reaction.reactionmeddrapt.exact': 'PNEUMONIA'}, 'count': 'serious'}
		fetching init/int/react data for {'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'SYMBICORT', 'patient.drug.medicinalproduct.exact': 'OMEPRAZOLE.', 'patient.reaction.react

		fetching init/int/react data for {'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'SYMBICORT', 'patient.drug.medicinalproduct.exact': 'XOLAIR', 'patient.reaction.reactionmeddrapt.exact': 'COUGH'}, 'count': 'serious'}
		fetching init/int/react data for {'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'SYMBICORT', 'patient.drug.medicinalproduct.exact': 'XOLAIR', 'patient.reaction.reactionmeddrapt.exact': 'WHEEZING'}, 'count': 'serious'}
		fetching init/int/react data for {'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'SYMBICORT', 'patient.drug.medicinalproduct.exact': 'XOLAIR', 'patient.reaction.reactionmeddrapt.exact': 'MALAISE'}, 'count': 'serious'}
		fetching init/int/react data for {'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'SYMBICORT', 'patient.drug.medicinalproduct.exact': 'XOLAIR', 'patient.reaction.reactionmeddrapt.exact': '

		fetching non-exact init/int/react data for ({'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'TAGRISSO', 'patient.drug.medicinalproduct.exact': 'IRESSA', 'patient.reaction.reactionmeddrapt': 'METASTASES TO CENTRAL NERVOUS SYSTEM'}, 'count': 'serious'})
		fetching init/int/react data for {'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'TAGRISSO', 'patient.drug.medicinalproduct.exact': 'IRESSA', 'patient.reaction.reactionmeddrapt.exact': 'DEATH'}, 'count': 'serious'}
		fetching init/int/react data for {'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'TAGRISSO', 'patient.drug.medicinalproduct.exact': 'IRESSA', 'patient.reaction.reactionmeddrapt.exact': 'RASH'}, 'count': 'serious'}
		fetching non-exact init/int/react data for ({'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'TAGRISSO', 'patient.drug.medicinalproduct.exact': 'IRESSA', 'patie

...

		fetching init/int/react data for {'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'SELOKEN', 'patient.drug.medicinalproduct.exact': 'ASPIRIN', 'patient.reaction.reactionmeddrapt.exact': 'PYREXIA'}, 'count': 'serious'}
		fetching init/int/react data for {'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'SELOKEN', 'patient.drug.medicinalproduct.exact': 'ASPIRIN', 'patient.reaction.reactionmeddrapt.exact': 'ANAEMIA'}, 'count': 'serious'}
		fetching non-exact init/int/react data for ({'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'SELOKEN', 'patient.drug.medicinalproduct.exact': 'ASPIRIN', 'patient.reaction.reactionmeddrapt': 'DRUG INTERACTION'}, 'count': 'serious'})
		fetching non-exact init/int/react data for ({'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'SELOKEN', 'patient.drug.medicinalproduct.exact': 'ASPIRIN', 'patient.reaction.rea

		fetching init/int/react data for {'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'SELOKEN', 'patient.drug.medicinalproduct.exact': 'TROMBYL', 'patient.reaction.reactionmeddrapt.exact': 'BRADYCARDIA'}, 'count': 'serious'}
		fetching non-exact init/int/react data for ({'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'SELOKEN', 'patient.drug.medicinalproduct.exact': 'TROMBYL', 'patient.reaction.reactionmeddrapt': 'CEREBRAL HAEMORRHAGE'}, 'count': 'serious'})
		fetching init/int/react data for {'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'SELOKEN', 'patient.drug.medicinalproduct.exact': 'TROMBYL', 'patient.reaction.reactionmeddrapt.exact': 'NAUSEA'}, 'count': 'serious'}
		fetching init/int/react data for {'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'SELOKEN', 'patient.drug.medicinalproduct.exact': 'TROMBYL', 'patient.reaction.reactio

		fetching init/int/react data for {'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'TOPROL-XL', 'patient.drug.medicinalproduct.exact': 'TOPROL XL', 'patient.reaction.reactionmeddrapt.exact': 'DIZZINESS'}, 'count': 'serious'}
		fetching init/int/react data for {'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'TOPROL-XL', 'patient.drug.medicinalproduct.exact': 'TOPROL XL', 'patient.reaction.reactionmeddrapt.exact': 'DIARRHOEA'}, 'count': 'serious'}
		fetching init/int/react data for {'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'TOPROL-XL', 'patient.drug.medicinalproduct.exact': 'TOPROL XL', 'patient.reaction.reactionmeddrapt.exact': 'HEADACHE'}, 'count': 'serious'}
		fetching non-exact init/int/react data for ({'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'TOPROL-XL', 'patient.drug.medicinalproduct.exact': 'TOPROL XL', 'patient.reacti

		fetching init/int/react data for {'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'TOPROL-XL', 'patient.drug.medicinalproduct.exact': 'ASPIRIN', 'patient.reaction.reactionmeddrapt.exact': 'PAIN'}, 'count': 'serious'}
		fetching init/int/react data for {'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'TOPROL-XL', 'patient.drug.medicinalproduct.exact': 'ASPIRIN', 'patient.reaction.reactionmeddrapt.exact': 'DIARRHOEA'}, 'count': 'serious'}
		fetching non-exact init/int/react data for ({'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'TOPROL-XL', 'patient.drug.medicinalproduct.exact': 'ASPIRIN', 'patient.reaction.reactionmeddrapt': 'DRUG INEFFECTIVE'}, 'count': 'serious'})
	fetching reactions data for (TOPROL-XL,PLAVIX)
		fetching non-exact init/int/react data for ({'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'TOPROL-XL', 'patient.drug.me

		fetching init/int/react data for {'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'TOPROL-XL', 'patient.drug.medicinalproduct.exact': 'SYNTHROID', 'patient.reaction.reactionmeddrapt.exact': 'FALL'}, 'count': 'serious'}
	fetching reactions data for (TOPROL-XL,NEXIUM)
		fetching non-exact init/int/react data for ({'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'TOPROL-XL', 'patient.drug.medicinalproduct.exact': 'NEXIUM', 'patient.reaction.reactionmeddrapt': 'CHRONIC KIDNEY DISEASE'}, 'count': 'serious'})
		fetching non-exact init/int/react data for ({'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'TOPROL-XL', 'patient.drug.medicinalproduct.exact': 'NEXIUM', 'patient.reaction.reactionmeddrapt': 'RENAL FAILURE'}, 'count': 'serious'})
		fetching init/int/react data for {'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'TOPROL-XL', 'patient.dr

In [87]:
df.to_csv('../data/interim/extracted_data.csv',index=False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

plt.savefig('books_read.png')
%matplotlibotlib inline%

In [81]:
df

,term,count,initial_drug,interaction_drug,reaction
0,1,6021,SYMBICORT,SYMBICORT,DYSPNOEA
1,2,4949,SYMBICORT,SYMBICORT,DYSPNOEA
0,1,4486,SYMBICORT,SYMBICORT,ASTHMA
1,2,1255,SYMBICORT,SYMBICORT,ASTHMA
0,1,2637,SYMBICORT,SYMBICORT,COUGH
...,...,...,...,...,...
1,2,2,TAGRISSO,DEXAMETHASONE.,VOMITING
0,1,6,TAGRISSO,DEXAMETHASONE.,FATIGUE
1,2,1,TAGRISSO,DEXAMETHASONE.,FATIGUE
0,1,4,TAGRISSO,DEXAMETHASONE.,RASH


In [84]:
init_int_react_params

{'search': {'receivedate': '[20040101+TO+20201024]',
  'patient.drug.medicinalproduct': 'TAGRISSO',
  'patient.drug.medicinalproduct.exact': 'DEXAMETHASONE.',
  'patient.reaction.reactionmeddrapt.exact': 'DECREASED APPETITE'},
 'count': 'serious'}

In [85]:
[res.update({'initial_drug':initial_drug,'interaction_drug':interaction_drug,'reaction':reaction})\
                                         for res in init_int_react_results]

[None, None]

In [68]:
faersApi.make_call(_params_10_reacts_).json()#['results']

{'meta': {'disclaimer': 'Do not rely on openFDA to make decisions regarding medical care. While we make every effort to ensure that data is accurate, you should assume all results are unvalidated. We may limit or otherwise restrict your access to the API in line with our Terms of Service.',
  'terms': 'https://open.fda.gov/terms/',
  'license': 'https://open.fda.gov/license/',
  'last_updated': '2020-10-24'},
 'results': [{'term': 'DEATH', 'count': 49},
  {'term': 'DIARRHOEA', 'count': 16},
  {'term': 'MALIGNANT NEOPLASM PROGRESSION', 'count': 11},
  {'term': 'NAUSEA', 'count': 8},
  {'term': 'VOMITING', 'count': 8},
  {'term': 'FATIGUE', 'count': 7},
  {'term': 'RASH', 'count': 7},
  {'term': 'DECREASED APPETITE', 'count': 6},
  {'term': 'OFF LABEL USE', 'count': 6},
  {'term': 'CONDITION AGGRAVATED', 'count': 5}]}

In [77]:
#SYMBICORT,SYMBICORT,DYSPNOEA
faersApi.make_call({'search': {'receivedate': '[20040101+TO+20201024]',
  'patient.drug.medicinalproduct': 'SYMBICORT',
  'patient.drug.medicinalproduct.exact': 'SYMBICORT',
  'patient.reaction.reactionmeddrapt.exact': 'DYSPNOEA'},
 'count': 'serious'}).json()

{'meta': {'disclaimer': 'Do not rely on openFDA to make decisions regarding medical care. While we make every effort to ensure that data is accurate, you should assume all results are unvalidated. We may limit or otherwise restrict your access to the API in line with our Terms of Service.',
  'terms': 'https://open.fda.gov/terms/',
  'license': 'https://open.fda.gov/license/',
  'last_updated': '2020-10-24'},
 'results': [{'term': 1, 'count': 6021}, {'term': 2, 'count': 4949}]}

In [60]:
init_int_react_params

{'search': {'receivedate': '[20040101+TO+20201024]',
  'patient.drug.medicinalproduct': 'SYMBICORT',
  'patient.drug.medicinalproduct.exact': 'VENTOLIN',
  'patient.reaction.reactionmeddrapt.exact': 'RENAL INJURY',
  'patient.reaction.reactionmeddrapt': 'FATIGUE'},
 'count': 'serious'}

In [42]:
df

,term,count,initial_drug,interaction_drug,reaction
0,1,6021,SYMBICORT,SYMBICORT,DYSPNOEA
1,2,4949,SYMBICORT,SYMBICORT,DYSPNOEA
0,1,4486,SYMBICORT,SYMBICORT,ASTHMA
1,2,1255,SYMBICORT,SYMBICORT,ASTHMA
0,1,2637,SYMBICORT,SYMBICORT,COUGH
...,...,...,...,...,...
1,2,140,NEXIUM,NEXIUM 24HR,PAIN
0,1,1117,NEXIUM,DEXILANT,DEATH
0,1,625,NEXIUM,DEXILANT,HYPERCHLORHYDRIA
1,2,46,NEXIUM,DEXILANT,HYPERCHLORHYDRIA


In [31]:
faersApi.make_call(init_int_react_params).json()

{'meta': {'disclaimer': 'Do not rely on openFDA to make decisions regarding medical care. While we make every effort to ensure that data is accurate, you should assume all results are unvalidated. We may limit or otherwise restrict your access to the API in line with our Terms of Service.',
  'terms': 'https://open.fda.gov/terms/',
  'license': 'https://open.fda.gov/license/',
  'last_updated': '2020-10-24'},
 'results': [{'term': 1, 'count': 6021}, {'term': 2, 'count': 4949}]}

In [32]:
init_int_react_results

[None, None]

In [30]:
init_int_react_params

{'search': {'receivedate': '[20040101+TO+20201024]',
  'patient.drug.medicinalproduct': 'SYMBICORT',
  'patient.drug.medicinalproduct.exact': 'SYMBICORT',
  'patient.reaction.reactionmeddrapt.exact': 'DYSPNOEA'},
 'count': 'serious'}

In [ ]:
# Cleanup
df = df.reset_index(drop=True).rename({'term':'serious'},axis=1)
# Make the 'serious' column more useful, e.g. make 'Not Serious' into 0, keep 'Serious' as 1
df.loc[df['serious']==2,'serious'] = 0
        


In [25]:
interaction_results

[{'term': 'NAUSEA', 'count': 3913},
 {'term': 'FATIGUE', 'count': 3514},
 {'term': 'DYSPNOEA', 'count': 3421},
 {'term': 'DIARRHOEA', 'count': 3142},
 {'term': 'DIZZINESS', 'count': 3125},
 {'term': 'DRUG INEFFECTIVE', 'count': 3101},
 {'term': 'PAIN', 'count': 2721},
 {'term': 'ASTHENIA', 'count': 2719},
 {'term': 'HEADACHE', 'count': 2687},
 {'term': 'HYPERTENSION', 'count': 2543}]

In [18]:
resp.json()['results']

[{'term': 'TOPROL-XL', 'count': 23346},
 {'term': 'TOPROL XL', 'count': 19054},
 {'term': 'WELLBUTRIN XL', 'count': 13098},
 {'term': 'LIPITOR', 'count': 8344},
 {'term': 'ASPIRIN', 'count': 8093},
 {'term': 'PLAVIX', 'count': 6184},
 {'term': 'LASIX', 'count': 6125},
 {'term': 'SYNTHROID', 'count': 5762},
 {'term': 'NEXIUM', 'count': 5642},
 {'term': 'ASPIRIN.', 'count': 5387},
 {'term': 'NORVASC', 'count': 4787}]

In [7]:
results_list = []
failed_drug_queries = []
for drug in A_Z_drugs:
    #Get top 10 interactions
    params_10_interacts['search'].update({'patient.drug.medicinalproduct':drug})
    print(params_10_interacts)
    try:
        results = faersApi.make_call(params_10_interacts).json()['results']
        print(f'fetching data for {drug}')
    except KeyError:
        # This occurs when a search for a given drug fails to return any results
        #failed_drug_queries.append(drug)
        #print(f'{drug} did not return results')
        # Try the lowercase version of the drug as well, then move along
        try:
            params_10_interacts['search'].update({'patient.drug.medicinalproduct':drug.lower()})
            results = faersApi.make_call(params_10_interacts).json()['results']
            print(f'fetching data for {drug}')
        except KeyError:
            # This occurs when a search for a given drug fails to return any results
            failed_drug_queries.append(drug)
            print(f'{drug} did not return results')
            continue
    for res in results:
        params_10_ = copy(params_10)
        params_10_['search'].update({'patient.drug.medicinalproduct':drug})
        if drug.lower() == res['term'].lower():
            pass
        else:
            params_10_['search'].update({'patient.drug.medicinalproduct.exact':res['term']})
            #print(params_10_)
            res_list = faersApi.return_categorical_counts(query_params=params_10_
                                                          , categories=faers_translations
                                                          , count_by='patient.reaction.reactionmeddrapt.exact'
                                                          , labels = {'timeframe':'all','primary_drug':drug,'interaction':res['term']})
            results_list.extend(res_list)
        

{'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'SYMBICORT'}, 'count': 'patient.drug.medicinalproduct.exact', 'limit': 11}
fetching data for SYMBICORT
{'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'TAGRISSO'}, 'count': 'patient.drug.medicinalproduct.exact', 'limit': 11}
TAGRISSO did not return results
{'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'NEXIUM'}, 'count': 'patient.drug.medicinalproduct.exact', 'limit': 11}
NEXIUM did not return results
{'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'CRESTOR'}, 'count': 'patient.drug.medicinalproduct.exact', 'limit': 11}
CRESTOR did not return results
{'search': {'receivedate': '[20040101+TO+20201024]', 'patient.drug.medicinalproduct': 'FARXIGA'}, 'count': 'patient.drug.medicinalproduct.exact', 'limit': 11}
FARXIGA did not return results
{'search': {'receivedate': '[20040101+TO+20201024]'

In [11]:

resp = faersApi.make_call(params_10_interacts)


In [12]:
resp.url

'https://api.fda.gov/drug/event.json?api_key=AxJ349dp7cWgZrO4dYC7mk3M4cTqHqa4n375Fao0&search=receivedate:%5B20040101+TO+20201024%5D+AND+patient.drug.medicinalproduct:toprol-xl&count=patient.drug.medicinalproduct.exact&limit=11'

In [13]:
resp.json()

{'meta': {'disclaimer': 'Do not rely on openFDA to make decisions regarding medical care. While we make every effort to ensure that data is accurate, you should assume all results are unvalidated. We may limit or otherwise restrict your access to the API in line with our Terms of Service.',
  'terms': 'https://open.fda.gov/terms/',
  'license': 'https://open.fda.gov/license/',
  'last_updated': '2020-10-24'},
 'results': [{'term': 'TOPROL-XL', 'count': 23346},
  {'term': 'TOPROL XL', 'count': 19054},
  {'term': 'WELLBUTRIN XL', 'count': 13098},
  {'term': 'LIPITOR', 'count': 8344},
  {'term': 'ASPIRIN', 'count': 8093},
  {'term': 'PLAVIX', 'count': 6184},
  {'term': 'LASIX', 'count': 6125},
  {'term': 'SYNTHROID', 'count': 5762},
  {'term': 'NEXIUM', 'count': 5642},
  {'term': 'ASPIRIN.', 'count': 5387},
  {'term': 'NORVASC', 'count': 4787}]}

In [ ]:
results_list = []
for res in results:
    params_1000_ = copy(params_1000)
    params_1000_['search'].update({'patient.drug.medicinalproduct':drug})
    if drug.lower() == res['term'].lower():
        pass
    else:
        params_1000_['search'].update({'patient.drug.medicinalproduct.exact':res['term']})
        print(params_1000_)
        res_list = faersApi.return_categorical_counts(query_params=params_1000_
                                                      , categories=faers_translations
                                                      , count_by='patient.reaction.reactionmeddrapt.exact'
                                                      , labels = {'timeframe':'all','primary_drug':drug,'interaction':res['term']})
        results_list.extend(res_list)
df = pd.DataFrame.from_records(results_list)
# Get reactions that occurred at least 10 times
for term in df.groupby('term')['count'].sum()[df.groupby('term')['count'].sum()>10].index:

In [ ]:
df = pd.DataFrame.from_records(results_list)

In [ ]:
df[df['term'] in (df['term'].value_counts()>1).index]

In [ ]:
df['term'].value_counts()[df['term'].value_counts()>3]

In [ ]:
for term in df.groupby('term')['count'].sum()[df.groupby('term')['count'].sum()>10].index:
    print(term)